<a href="https://colab.research.google.com/github/judith182/Estadistica-y-probabilidad/blob/main/Reclamaciones_p%C3%B3lizas_de_seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###MetodosAproximacionParaS
library(actuar)
library(fitdistrplus)
library(vcd)
library(ADGofTest)
library(mixtools)
library(mclust)
library(MASS)

###Datos
###Base de datos para perdidas agregadas

base<-read.csv("C:/Users/judith/Desktop/Cursos2021-I/Riesgo2021/PerdidasAgregadas.csv")
head(base)
attach(base)

###La variable frecuencia guarda la frecuencia de reclamaciones de estas 200 polizas

barplot(table(frecuencia),col=rainbow(15),main="Tabla de frecuencias de reclamacion",col.main="darkblue")

###Ajuste modelo discreto: Poisson

disc.fit<-goodfit(frecuencia,type = "poisson", method = "ML")
plot(disc.fit)
summary(disc.fit)
disc.fit$par

###Entonces, el numero medio de reclamaciones es: 4.925

###Ajuste distribucion continua: montos de reclamacion

###Guardando los montos de reclamacion en un vector y eliminando las "NA"

severidad<-c(base[,2],base[,3],base[,4],base[,5],base[,6],base[,7],base[,8],base[,9],base[,10],base[,11],base[,12],base[,13])
severidad1<-severidad[!is.na(severidad)]
length(severidad1)
severidad1[1:50]

plot(density(severidad1))


### Posibles densidades asociadas (no todas)

descdist(severidad1,discrete=FALSE,boot=5000,obs.pch = 19, boot.col="darkviolet")

### Mi funcion (Recuerden, ustedes no la tienen)

source("C:/Users/Salvador/Desktop/Cursos2021-I/Riesgo2021/autodistfit.R")

fitData(severidad1,c("gamma","weibull","lognormal"),sample=1)

fG<-fitdist(severidad1,"gamma")

ks.test(severidad1,"pgamma",shape=fG$estimate[1],rate=fG$estimate[2])
ad.test(severidad1, pgamma,shape=fG$estimate[1],rate=fG$estimate[2])

plot(fG)

### Entonces: N~Poisson(4.925) y X~Gamma(forma=50.98746956,rate=0.02906342). rate=1/scale, scale=34.40751

########################################################################################################################################
########################################### APROXIMACION NORMAL ########################################################################

muN<-4.925; muX<-mgamma(1,shape=fG$estimate[1],rate=fG$estimate[2]);varN<-muN;varX<-mgamma(2,shape=fG$estimate[1],rate=fG$estimate[2])-muX^2
muN;muX;varN;varX

ES<-muN*muX;VS<-muN*mgamma(2,shape=fG$estimate[1],rate=fG$estimate[2])

ES;VS

Z<-function(s){pnorm(s,ES,sqrt(VS))}

curve(Z,from=0,to=25000,main="Aproximacion: Distribucion normal",lwd=2,col.main="darkblue",col="magenta")

1-Z(20000)

Z1<-function(s){dnorm(s,ES,sqrt(VS))}
curve(Z1,from=-5000,to=25000,main="Aproximacion: Densidad normal",lwd=2,col.main="darkred",col="darkmagenta")

Z(ES)

qnorm(0.95,ES,sqrt(VS));qnorm(0.99,ES,sqrt(VS));qnorm(0.995,ES,sqrt(VS))

qnorm(0.90,ES,sqrt(VS))

###Usando actuar (Aproximacion Normal)

AN<-aggregateDist("normal",moments=c(ES,VS), x.scale=1)

VaR(AN);VaR(AN,0.995)


###Si no hubieramos logrado ajustar modelos a nuestros datos

meanX<-mean(severidad1);varX<-var(severidad1);meanN<-mean(frecuencia);varN<-var(frecuencia)

E.S<-meanX*meanN;V.S<-meanN*varX+varN*meanX^2

E.S;V.S

Z2<-function(s){pnorm(s,E.S,sqrt(V.S))}

curve(Z2,from=0,to=25000,main="Aproximacion: Distribucion normal",lwd=2,col.main="darkblue",col="magenta")

Z3<-function(s){dnorm(s,E.S,sqrt(V.S))}
curve(Z3,from=-5000,to=25000,main="Aproximacion: Densidad normal",lwd=2,col.main="darkred",col="darkmagenta")

###Comparacion Aproximacion Normal vs. Aproximacion Normal Empirica

curve(Z,from=-5000,to=25000,main="Comparaciones: Aproximacion normal vs. Aproximacion normal empirica",lwd=2,col.main="darkred",col="darkmagenta")
curve(Z2,from=0,to=25000,col="magenta",xlab="",ylab="",add=TRUE,lwd=2)
legend(locator(1),legend=c("Aproximacion Normal","Aproximacion Normal Empirica"),col=c("darkmagenta","magenta"),bty="n",lwd=2)

curve(Z1,from=-5000,to=25000,main="Comparaciones: Aproximacion normal vs. Aproximacion normal empirica",lwd=2,col.main="darkred",col="darkmagenta")
curve(Z3,from=0,to=25000,col="magenta",xlab="",ylab="",add=TRUE,lwd=2)
legend(locator(1),legend=c("Aproximacion Normal","Aproximacion Normal Empirica"),col=c("darkmagenta","magenta"),bty="n",lwd=2)

#############################################################################################################################################
############################################# APROXIMACION LOG-NORMAL #######################################################################
###Mismos datos anteriores. N~Poisson(4.925) y X~Gamma(forma=50.98746956,rate=0.02906342). rate=1/scale, scale=34.40751
###El sistema de ecuaciones es 2*log(E(S))=2mu+sigma^2   y  log(E(S^2))=2mu+2sigma^2
### Ya tenemos E(S) y E(S^2)=Var(S)+E(S)^2
### Resolvamos el sistema apoyandonos en R

A=matrix(c(2,1,2,2),2,byrow=TRUE)
A
b<-c(2*log(ES),log(VS+ES^2))
sol<-solve(A,b)

Z4<-function(s){pnorm(log(s),sol[1],sqrt(sol[2]))}
curve(Z4,from=0.01,to=25000,main="Aproximacion: Distribucion log-normal",lwd=2,col.main="darkred",col="darkmagenta")

Z5<-function(s){dnorm(log(s),sol[1],sqrt(sol[2]))}
curve(Z5,from=0.01,to=25000,main="Aproximacion: Densidad log-normal",lwd=2,col.main="darkred",col="darkmagenta")

###Aproximacion log normal empirica

B=matrix(c(2,1,2,2),2,byrow=TRUE)
B
c<-c(2*log(E.S),log(V.S+E.S^2))
sol1<-solve(B,c)

Z41<-function(s){pnorm(log(s),sol1[1],sqrt(sol1[2]))}
curve(Z41,from=0.01,to=25000,main="Aproximacion: Distribucion log-normal",lwd=2,col.main="darkred",col="darkmagenta")

Z51<-function(s){dnorm(log(s),sol1[1],sqrt(sol1[2]))}
curve(Z51,from=0.01,to=25000,main="Aproximacion: Densidad log-normal",lwd=2,col.main="darkred",col="darkmagenta")


curve(Z4,from=0.01,to=25000,main="Aproximacion: Distribucion log-normal",lwd=2,col.main="darkred",col="darkmagenta")
curve(Z41,from=0.01,to=25000,lwd=2,col="darkblue",add=TRUE,xlab="",ylab="")
legend(locator(1),legend=c("Aproximacion Log-Normal","Aproximacion Log-normal emp�rica"),col=c("darkmagenta","darkblue"),bty="n",lwd=2)


###Comparando con las aproximaciones normales anteriores

curve(Z,from=-5000,to=25000,main="Comparaciones: Aproximacion normal vs. Aproximacion normal empirica vs. Aproximacion Log-normal",lwd=2,col.main="darkred",col="darkmagenta")
curve(Z2,from=0,to=25000,col="magenta",xlab="",ylab="",add=TRUE,lwd=2)
curve(Z4,from=0.01,to=25000,col="darkblue",xlab="",ylab="",add=TRUE,lwd=2)

legend(locator(1),legend=c("Aproximacion Normal","Aproximacion Normal Empirica","Aproximacion Log-normal"),col=c("darkmagenta","magenta","darkblue"),bty="n",lwd=2)


###Aproximacion gamma transladada. Observemos que la distribucion de nuestros montos !!!es una gamma!!!
###Ya que la distribucion discreta es una Poisson, entonces la distribucion de S es Poisson compuesta
###y el sesgo de S, se calcula como lambda*E(X^3)/VS^(3/2)

mu<-ES;sigma2<-VS;tau<-meanN*mgamma(3,shape=fG$estimate[1],rate=fG$estimate[2])/VS^(3/2)
mu;sigma2;tau
alpha<-4/tau^2;theta<-sqrt(sigma2)*tau/2;k<-mu-2*sqrt(sigma2)/tau
alpha;theta;k

Z6<-function(s){pgamma(s-k,shape=alpha,scale=theta)}
curve(Z6,from=0,to=25000,main="Aproximacion: Distribucion gamma transladada",lwd=2,col.main="darkred",col="darkmagenta")

Z7<-function(s){dgamma(s-k,shape=alpha,scale=theta)}
curve(Z7,from=0,to=25000,main="Aproximacion: Densidad gamma transladada",lwd=2,col.main="darkred",col="darkmagenta")


###Comparacion de todas las aproximaciones
curve(Z,from=-5000,to=25000,main="Comparaciones: Aproximacion normal vs. Aproximacion normal empirica vs. Aproximacion Log-normal",lwd=2,col.main="darkred",col="darkmagenta")
curve(Z2,from=0,to=25000,col="magenta",xlab="",ylab="",add=TRUE,lwd=2)
curve(Z4,from=0.01,to=25000,col="darkblue",xlab="",ylab="",add=TRUE,lwd=2)
curve(Z6,from=0,to=25000,col="darkorange",add=TRUE,lwd=2)
legend(locator(1),legend=c("Aproximacion Normal","Aproximacion Normal Empirica","Aproximacion Log-normal","Gamma transladada"),col=c("darkmagenta","magenta","darkblue","darkorange"),bty="n",lwd=2)

### Convoluciones
### Convolucion de dos distribuciones Poisson
### Sabemos que la suma (convolucion) de dos v.s.a.s Poisson (lambda) es otra Poisson con par�metro lambda1+lambda2

x<-dpois(seq(0,20,1),2)

y<-dpois(seq(0,20,1),2)

z<-convolve(x,rev(y),type="o")  ###La convolucion de las dos Poisson

w<-dpois(seq(0,40,1),4)  ###La distribucion teorica

v<-as.matrix(cbind(z[1:20],w[1:20]))

barplot(t(v),beside=TRUE,col=c("blue","red"),legend.text=c("Convolucion","Distribucion teorica"))

### Convolucion de dos exponenciales 0.1 (media=10)
### Sabemos que la suma (convolucion) de dos v.s.a.s exponenciales (alpha, lambda) es otra una gamma con parametro de
### forma alpha1+alpha2 y parametro de escala lambda
### Para usar la funcion convolve necesitamos discretizar estas distribuciones continuas

x1<-discretize(pexp(x,0.1), method = "rounding", from = 0, to = 20,step=0.1)

x2<-discretize(pexp(x,0.1), method = "rounding", from = 0, to = 20,step=0.1)

x3<-convolve(x1,rev(x2),type="o")

x4<-discretize(pgamma(x,2,0.1), method = "rounding", from = 0, to = 20,step=0.1)

x5<-as.matrix(cbind(x3[1:20],x4[1:20]))

barplot(t(x5),beside=TRUE,col=c("green","violet"),legend.text=c("Convolucion","Distribucion teorica"))

### Nuestro ejemplo de notas
################### convolucion

fr<-c(0.1,0.2,0.3,0.4)  ###Igualamos los vectores con ceros donde sea necesario
fs<-c(0,0.4,0.6,0)      ###Utilizando la definicion de densidad: "cero en cualquier otro lado"

Fs<-aggregateDist("convolution", model.freq = fr, model.sev = fs)

quantile(Fs)

plot(Fs,col="blue",main="Distribucion de S: Metodo convolucion",col.main="red",col.lab="green",sub="",xlab="s",ylab=expression(F[S](s)))

win.graph()

plot(Fs, do.points = FALSE, verticals = TRUE, xlim = c(0, 5),col="darkblue",lwd=2)

CDFs<-Fs(c(0,1,2,3,4,5,6))

dfs<-diff(c(0,CDFs))

barplot(dfs,col="darkred",main="Densidad de S: Metodo convoluci�n",col.main="darkblue",col.lab="darkgreen",sub="",)

par(mfrow=c(1,2))

plot(Fs,col="blue",main="Distribucion de S: Metodo convolucion",col.main="red",col.lab="green",sub="")

barplot(dfs,col="darkred",main="Densidad de S: Metodo convolucion",col.main="darkblue",col.lab="darkgreen",sub="")


VaR(Fs)

VaR(Fs,0.995)

CTE(Fs)

CTE(Fs,0.995)

### Datos reales (Seguro dental)

### Ejemplo convolucion de distribuciones discretas (tablas)
### Observese que los recorridos de X y de N no son iguales
### X no toma valor en cero y N si. X toma valores en 9 y 10, y N, no
### Debemos de completar con CEROS en cada caso, para igualar los recorridos

### Los originales

x1<-seq(1,10,1)

x2<-c(0.150,0.200,0.250,0.125,0.075,0.050,0.050,0.050,0.025,0.025)

x<-data.frame(cbind(x2,x1))

n1<-seq(0,8,1)

n2<-c(0.05,0.10,0.15,0.20,0.25,0.15,0.06,0.03,0.01)

n<-data.frame(cbind(n2,n1))

###Los "completados"

fs<-c(0,x2)   # vector de probabilidades asociado a la severidad (observese que aumentamos el valor cero)

fr<-c(n2,0,0)#vector de probabilidades asociado a la frecuencia (observese que aumentamos dos ceros para que los vectores sean de igual tamanio)

Fs<-aggregateDist("convolution", model.freq = fr, model.sev = fs)

quantile(Fs)

plot(Fs,col="darkblue",main="Funcion de distribucion seguro dental",col.main="darkred",col.lab="darkgreen",sub="",xlab="s",ylab=expression(F[S](s)))

win.graph()

plot(Fs, do.points = FALSE, verticals = TRUE,col="darkblue",main="Funcion de distribucion seguro dental",col.main="darkred",
     col.lab="darkgreen",xlab="s",ylab=expression(F[S](s)))

CDFs<-Fs(c(seq(0,100,1)))

dfs<-diff(c(0,CDFs))

barplot(dfs,col="orange",main="Funcion de distribucion de S via convolucion",xlab="s",ylab=expression(f[S](s)),col.main="darkred",
        col.lab="darkgreen")

VaR(Fs)

VaR(Fs,0.995)

CTE(Fs)

CTE(Fs,0.995)

### Tienen sentido estos valores?. "La neta, NO".
### Como deberia de hacerse?

Fs<-aggregateDist("convolution", model.freq = fr, model.sev = fs,x.scale=25)

quantile(Fs)

plot(Fs,col="darkblue",main="Funcion de distribucion seguro dental",col.main="darkred",col.lab="darkgreen",sub="")

win.graph()

plot(Fs, do.points = FALSE, verticals = TRUE,col="darkblue",main="Funcion de distribucion seguro dental",col.main="darkred",

     col.lab="darkgreen")

CDFs<-Fs(c(seq(0,1000,25)))

dfs<-diff(c(0,CDFs))

barplot(dfs,col="orange",main="Funcion de distribucion de S via convolucion",xlab="s",ylab=expression(f[S](s)),col.main="darkred",
        col.lab="darkgreen")

VaR(Fs)

VaR(Fs,0.995)

CTE(Fs)

CTE(Fs,0.995)

###PANJER notas
### Ejemplo Panjer

###X~BN(5,0.4) p=1/(1+beta)
###N~Poisson(2)

x1<-seq(0,100)
fx1<-dnbinom(x1,5,0.4)

###recursive==Panjer

Fs1<- aggregateDist("recursive",model.freq="poisson", model.sev=fx1,lambda=2)

1-Fs1(3)

S1<-Fs1(knots(Fs1))
fS1<-diff(c(0,S1))
plot(Fs1, main="Distribucion de Reclamaciones Agregadas: Panjer",col="darkblue",col.main="red",sub="",
xlab= "Aproximaci�n M�todo Recursivo (Panjer)",col.lab="darkgreen")

barplot(fS1,main="Densidad de Reclamaciones Agregadas: Panjer",col="darkgreen",col.main="red",sub="",
xlab= "Aproximacion Metodo Recursivo (Panjer)",col.lab="darkred",ylab=expression(f[S](x)))

par(mfrow=c(1,2))

plot(Fs1, main="Distribucion de Reclamaciones Agregadas: Panjer",col="darkblue",col.main="red",sub="",
xlab= "Aproximacion Metodo Recursivo (Panjer)",col.lab="darkgreen")

barplot(fS1,main="Densidad de Reclamaciones Agregadas: Panjer",col="darkgreen",col.main="red",sub="",
xlab= "Aproximacion Metodo Recursivo (Panjer)",col.lab="darkred",ylab=expression(f[S](x)))

VaR(Fs1)

VaR(Fs1,0.995)

CTE(Fs1)

CTE(Fs1,0.995)

###La tabla

cbind(s=seq(0,10),fS=fS1[1:11],FS=S1[1:11])

#########################################################################
###Pajer: Seguro dental

### Datos reales

x1<-seq(1,10,1)

x2<-c(0.150,0.200,0.250,0.125,0.075,0.050,0.050,0.050,0.025,0.025)

x<-data.frame(cbind(x2,x1))

n1<-seq(0,8,1)

n2<-c(0.05,0.10,0.15,0.20,0.25,0.15,0.06,0.03,0.01)

n<-data.frame(cbind(n2,n1))

fs<-c(0,x2)   # vector de probabilidades asociado a la severidad (observese que aumentamos el valor cero)

fr<-c(n2,0,0)#vector de probabilidades asociado a la frecuencia (observese que aumentamos dos ceros para que los vectores sean de igual tama�o)

poisfit1<-goodfit(n, type = "poisson", method = "ML")

summary(poisfit1)
plot(poisfit1)
poisfit1$par

###Curiosida'

poisfit2<-goodfit(x, type = "poisson",method = "ML")
summary(poisfit2)
plot(poisfit2)
poisfit2$par


### Modelo de perdidas agregadas utilizando Panjer

Fs2<-aggregateDist("recursive", model.freq = "poisson", model.sev=fs, lambda=3.4, x.scale=25)

plot(Fs2,main="Distribucion de Reclamaciones Agregadas",col="darkblue",col.main="red",sub="",
xlab= "Aproximacion Metodo Recursivo (Panjer)",col.lab="darkgreen")

FCDFs2<-Fs2(c(seq(0,1000,1)))

fCDFs2<-diff(c(0,FCDFs2))

barplot(fCDFs2,main="Densidad de Reclamaciones Agregadas",col="darkgreen",col.main="red",sub="",
xlab= "Aproximacion Metodo Recursivo (Panjer)",col.lab="darkred",col.lab="darkblue",ylab=expression(f[S](x)))

VaR(Fs2)

VaR(Fs2,0.995)

CTE(Fs2)

CTE(Fs2,0.995)

###Mismas cantidades, pero con el metodo de convolucion

Fs<-aggregateDist("convolution", model.freq = fr, model.sev = fs)

VaR(Fs)*25

VaR(Fs,0.995)*25

CTE(Fs)*25

CTE(Fs,0.995)*25

###Metodo de redondeo
###Ejemplo notas: pareto(4,50)

fx1<-discretize(ppareto(x,shape=4,scale=50), method = "rounding", from = 0, to = 10000,step=0.9)
sum(fx1)
cbind(x<-seq(0,10),f<-fx1[1:11])

###Comparacion

par(oma = c( 0, 0, 5, 0 ),col="darkblue")
par(mfrow=c(1,2))

fx<- discretize(ppareto(x, shape=4, scale=50), method="rounding",from=0,to=100,step=0.9)
v<- c(0:110)
plot(stepfun(v, diffinv(fx)),xlab="x",ylab=expression(F[X](x)),col="darkred",main="Comparacion: Funciones de distribucion",
     col.main="red",col.lab="darkgreen")
curve( ppareto(x, shape=4, scale=50),from=0,to=120,col="blue",add=TRUE)
t<- c("Real", "Discretizacion")
legend(20, 0.4, paste(t), lty=1,col=c("blue", "darkred"))


barplot(fx,xlab="x",ylab=expression(f[X](x)),col="green",main="Comparacion: Funciones de densidad",
        col.main="red",col.lab="darkgreen")
curve(dpareto(x, shape=4, scale=50),from=0,to=120,col="blue",add=TRUE)
t<- c("Real", "Discretizacion")
legend(20, 0.04, paste(t), lty=1, col=c("blue", "green"))

title("Comparacion: Pareto(4,50) continua vs. discretizada",,outer = TRUE)


### Metodo de simulacion
###Ejemplo: Poisson-BinomialNegativa

model.freq<-expression(data=rpois(2))
model.sev<-expression(data=rnbinom(size=5,prob=0.4))
SimF<-aggregateDist("simulation",nb.simul=10000,model.freq,model.sev)

plot(SimF, main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkblue",col.main="red",sub="",
     xlab= "Metodo de simulacion",col.lab="darkgreen")

plot(SimF,do.points=FALSE,verticals=TRUE,main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkblue",col.main="red",sub="",
     xlab="Metodo de simulacion",col.lab="darkgreen")

val<-SimF(knots(SimF))

Simf<-diff(c(0,val))

par(mfrow=c(1,2))

barplot(Simf, main="Densidad perdidas agregadas: Metodo de simulacion",col.lab="darkviolet",
        sub= "Aproximacion via simulacion",col="red",col.main="darkgreen",col.sub="skyblue",ylab=expression(f[S](s)))

plot(SimF, main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkgreen",col.main="red",sub="",xlab= "Metodo de simulacion",col.lab="darkgreen",ylab=expression(F[S](s)))

VaR(SimF)

CTE(SimF)

1-SimF(3)

###Con un modelo continuo para la severidad

model.freq<-expression(data=rpois(2))
model.sev<-expression(data=rpareto(shape=4,scale=50))
SimF<-aggregateDist("simulation",nb.simul=10000,model.freq,model.sev)

plot(SimF, main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkblue",col.main="red",sub="",
     xlab= "Metodo de simulacion",col.lab="darkgreen")

plot(SimF,do.points=FALSE,verticals=TRUE,main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkblue",col.main="red",sub="",
     xlab="Metodo de simulacion",col.lab="darkgreen")

val<-SimF(knots(SimF))

Simf<-diff(c(0,val))

par(mfrow=c(1,2))

barplot(Simf, main="Densidad perdidas agregadas: Metodo de simulacion",col.lab="darkviolet",
        sub= "Aproximacion via simulacion",col="red",col.main="darkgreen",col.sub="skyblue",ylab=expression(f[S](s)))

plot(SimF, main="Distribucion de Reclamaciones Agregadas (simulacion)",col="darkgreen",col.main="red",sub="",xlab= "Metodo de simulacion",col.lab="darkgreen",ylab=expression(F[S](s)))

VaR(SimF)

CTE(SimF)


###################################################################################################
###Comparando (que es gerundio) los metodos vistos utilizando los datos de seguro dental

x1<-seq(1,10,1)

x2<-c(0.150,0.200,0.250,0.125,0.075,0.050,0.050,0.050,0.025,0.025)

x<-data.frame(cbind(x2,x1))

n1<-seq(0,8,1)

n2<-c(0.05,0.10,0.15,0.20,0.25,0.15,0.06,0.03,0.01)

n<-data.frame(cbind(n2,n1))

###Los "completados"

fs<-c(0,x2)

fr<-c(n2,0,0)

ConSD<-aggregateDist("convolution", model.freq = fr, model.sev = fs)

PanSD<-aggregateDist("recursive", model.freq = "poisson", lambda=3.4,model.sev = fs)

model.freq<-expression(data=rpois(3.4))
model.sev<-expression(data=rpois(3.7))

SimSD<-aggregateDist("simulation",nb.simul=10000,model.freq,model.sev)
SimSD1<-SimSD(25*seq(0:100))

plot(ConSD,do.points=FALSE,verticals=TRUE,main="Comparacion: Convolucion, Panjer y Simulacion",col="darkred",col.main="red",sub="",xlab="x",col.lab="darkgreen")

plot(PanSD,do.points=FALSE,verticals=TRUE,main="",col="darkgreen", xlab=" ",add=T,sub="")

plot(SimSD,do.points=FALSE,verticals=TRUE,main="",col="darkorange",xlab="",sub="",add=T)

legend(locator(1),legend=c("Convolucion","Panjer","Simulacion"),bty="n",col=c("darkred","darkgreen","darkorange"),lty=1)

res<-matrix(c(25*VaR(ConSD),25*VaR(ConSD,0.995),25*CTE(ConSD),25*CTE(ConSD,0.995),25*VaR(PanSD),25*VaR(PanSD,0.995),25*CTE(PanSD),25*CTE(PanSD,0.995),25*VaR(SimSD),25*VaR(SimSD,0.995),
              25*CTE(SimSD),25*CTE(SimSD,0.995)),nrow=3,byrow=TRUE)
res

rownames(res)<-c("Convolucion","Panjer","Simulacion");colnames(res)<-c("VaR90","VaR95","VaR99","VaR99.5","TVaR90","TVaR95","TVaR99","TVaR99.5")
t(res)
